Importamos los datos desde kaggle, creando la carpeta de data, con los datos comprmidos ( como se descarga por defecto de kaggle), y ha continuacion descomprimimos las carpetas con las imagenes

In [5]:
import os

if not os.path.exists("data/raw"):
    os.makedirs("data/raw")
    !kaggle datasets download -d gunavenkatdoddi/eye-diseases-classification -p data/raw
    !unzip data/raw/eye-diseases-classification.zip -d data/raw


In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("gunavenkatdoddi/eye-diseases-classification")

# print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'